In [0]:
%sql
-- Set environment to data ingestion site
use catalog bronze;

-- Last item to be fixed customer and purchase must be accurate on a join for avocado_bronze

-- ========================================================
-- 1. Standardized Consumer with Validation Flags
-- ========================================================
create or replace temp view standardized_consumer as
select
      try_cast(NULLIF(trim(con.consumerid), '') as bigint) as consumer_id
    , initcap(trim(NULLIF(con.Sex, ''))) as sex
    , initcap(trim(NULLIF(con.ethnicity, ''))) as ethnicity
    , initcap(trim(NULLIF(con.Race, ''))) as race
    , try_cast(NULLIF(trim(con.Age), '') as bigint) as age
    , row_number() over (
        partition by try_cast(NULLIF(trim(con.consumerid), '') as bigint)
        order by try_cast(NULLIF(trim(con.Age), '') as int) desc
      ) as rnk -- All rnk columns are meant to visually show in data duplicates that exist
    , case when NULLIF(trim(con.consumerid), '') is null then 1 else 0 end as null_primary_key
from consumer_bronze con;

-- ========================================================
-- 2. Validated Consumer
-- ========================================================
create or replace temp view validated_consumer as
select distinct
    *
from standardized_consumer con
where con.null_primary_key = 0
;

-- ========================================================
-- 3. Standardized Purchase with Validation Flags
-- ========================================================
create or replace temp view standardized_purchase as
select
      try_cast(NULLIF(trim(pur.consumerid), '') as bigint) as consumer_id
    , try_cast(NULLIF(trim(pur.purchaseid), '') as bigint) as purchase_id
    , try_cast(NULLIF(trim(pur.graphed_date), '') as date) as graphed_date
    , try_cast(NULLIF(trim(pur.avocado_bunch_id), '') as bigint) as avocado_bunch_id
    , try_cast(NULLIF(trim(pur.reporting_year), '') as bigint) as reporting_year
    , initcap(trim(NULLIF(pur.`QA process`, ''))) as qa_process
    , try_cast(NULLIF(trim(pur.billing_provider_sku), '') as bigint) as billing_provider_sku
    , try_cast(NULLIF(trim(pur.grocery_store_id), '') as bigint) as grocery_store_id
    , try_cast(NULLIF(trim(pur.price_index), '') as double) as price_index
    , row_number() over (
        partition by try_cast(NULLIF(trim(pur.purchaseid), '') as bigint)
        order by try_cast(NULLIF(trim(pur.graphed_date), '') as date) desc
      ) as rnk
    -- Validation flags
    , case when try_cast(NULLIF(trim(pur.purchaseid), '') as bigint) is null 
          then 1 
          else 0
      end as invalid_purchase_id
    , case when try_cast(NULLIF(trim(pur.consumerid), '') as bigint) is null 
          then 1 
          else 0
      end as invalid_consumer_id
from purchase_bronze pur
;
-- ========================================================
-- 4. Validated Purchase
-- ========================================================
create or replace temp view validated_purchase as
select distinct
    pur.*
from standardized_purchase pur
inner join validated_consumer con on pur.consumer_id = con.consumer_id
where pur.invalid_purchase_id = 0
  and pur.invalid_consumer_id = 0
  ;
-- ========================================================
-- 5. Standardized Fertilizer with Validation Flags
-- ========================================================
create or replace temp view standardized_fertilizer as
select
      try_cast(NULLIF(trim(fer.purchaseid), '') as bigint) as purchase_id
    , try_cast(NULLIF(trim(fer.consumerid), '') as bigint) as consumer_id
    , try_cast(NULLIF(trim(fer.fertilizerid), '') as bigint) as fertilizer_id
    , initcap(trim(NULLIF(fer.type, ''))) as fertilizer_type
    , try_cast(NULLIF(trim(fer.mg), '') as bigint) as mg
    , trim(NULLIF(fer.frequency, '')) as frequency
    , row_number() over (
        partition by try_cast(NULLIF(trim(fer.fertilizerid), '') as bigint)
        order by try_cast(NULLIF(trim(fer.purchaseid), '') as bigint) desc
      ) as rnk
from fertilizer_bronze fer
;
-- ========================================================
-- 6. Validated Fertilizer
-- ========================================================
create or replace temp view validated_fertilizer as
select distinct
    fer.*
from standardized_fertilizer fer
inner join validated_purchase pur on fer.purchase_id = pur.purchase_id
inner join validated_consumer con on fer.consumer_id = con.consumer_id
where fer.purchase_id is not null
    and fer.consumer_id is not null
;
-- ========================================================
-- 7. Standardized Avocado with Validation Flags
-- ========================================================
create or replace temp view standardized_avocado as
select
      try_cast(NULLIF(trim(avo.consumerid), '') as bigint) as consumer_id
    , try_cast(NULLIF(trim(avo.purchaseid), '') as bigint) as purchase_id
    , try_cast(NULLIF(trim(avo.avocado_bunch_id), '') as bigint) as avocado_bunch_id
    , try_cast(NULLIF(trim(avo.PLU), '') as bigint) as PLU
    , try_cast(NULLIF(trim(avo.`ripe index when picked`), '') as bigint) as avocado_ripe_index
    , try_cast(NULLIF(trim(avo.born_date), '') as date) as born_date
    , try_cast(NULLIF(trim(avo.picked_date), '') as date) as picked_date
    , try_cast(NULLIF(trim(avo.sold_date), '') as date) as sold_date
    , row_number() over (
        partition by try_cast(NULLIF(trim(avo.purchaseid), '') as bigint)
        order by try_cast(NULLIF(trim(avo.born_date), '') as date) desc
      ) as rnk
    -- Validation flags
    , case when picked_date < born_date then 1 else 0 end as invalid_picked_date
    , case when sold_date < picked_date then 1 else 0 end as invalid_sold_date
from avocado_bronze avo
;
-- ========================================================
-- 8. Validated Avocado
-- ========================================================
create or replace temp view validated_avocado as
select avo.*
from standardized_avocado avo
inner join validated_purchase pur on avo.purchase_id = pur.purchase_id
inner join validated_consumer con on avo.consumer_id = con.consumer_id
where avo.invalid_picked_date = 0
  and avo.invalid_sold_date = 0
  ;
-- ========================================================
-- 9. Relationship Integrity Checks
-- ========================================================

-- Many consumers for one purchase
create or replace temp view invalid_purchase_multi_consumers as
select 
    purchase_id,
    collect_set(consumer_id) as consumers
from validated_purchase
group by purchase_id
having size(collect_set(consumer_id)) > 1;

-- Fertilizer ID tied to multiple purchases
create or replace temp view invalid_fertilizer_multi_purchases as
select 
    fertilizer_id,
    collect_set(purchase_id) as purchases
from validated_fertilizer
group by fertilizer_id
having size(collect_set(purchase_id)) > 1;

-- Avocados with the same purchase_id
create or replace temp view invalid_avocado_multi_purchases as
select 
    purchase_id
    , collect_set(consumer_id) as consumers
from validated_avocado
group by purchase_id
having size(consumers) > 1;

-- ========================================================
-- 10. Quarantine Table
-- ========================================================
create or replace table default.quarantine (
    source_table string,
    pk_value string,
    fk_values string,
    bad_column string,
    bad_value string,
    failure_reason string,
    full_row string
);

insert into default.quarantine
-- Consumer
select 'consumer', cast(consumer_id as string), null, 'consumer_id', cast(consumer_id as string), 'NULL PK', to_json(struct(*))
from standardized_consumer
where null_primary_key = 1

union all
-- Purchase PK/FK
select 'purchase', cast(purchase_id as string), concat('consumer_id=', cast(consumer_id as string)),
       'purchase_id', cast(purchase_id as string), 'NULL PK', to_json(struct(*))
from standardized_purchase where invalid_purchase_id = 1

union all
select 'purchase', cast(purchase_id as string), concat('consumer_id=', cast(consumer_id as string)),
       'consumer_id', cast(consumer_id as string), 'NULL FK', to_json(struct(*))
from standardized_purchase where invalid_consumer_id = 1

union all
-- Fertilizer FK
select 'fertilizer', cast(fertilizer_id as string), concat('consumer_id=', cast(consumer_id as string), ',purchase_id=', cast(purchase_id as string)),
       'purchase_id', cast(purchase_id as string), 'NULL FK', to_json(struct(*))
from standardized_fertilizer where purchase_id is null

union all
select 'fertilizer', cast(fertilizer_id as string), concat('consumer_id=', cast(consumer_id as string), ',purchase_id=', cast(purchase_id as string)),
       'consumer_id', cast(consumer_id as string), 'NULL FK', to_json(struct(*))
from standardized_fertilizer where consumer_id is null

union all
-- Avocado PK/FK
select 'avocado', cast(purchase_id as string), concat('consumer_id=', cast(consumer_id as string)),
       'picked_date', cast(picked_date as string), 'picked before born', to_json(struct(*))
from standardized_avocado where invalid_picked_date = 1

union all
select 'avocado', cast(purchase_id as string), concat('consumer_id=', cast(consumer_id as string)),
       'sold_date', cast(sold_date as string), 'sold before picked', to_json(struct(*))
from standardized_avocado where invalid_sold_date = 1

union all
-- Relationship violations
select 'purchase', cast(purchase_id as string),
       concat('consumers=', array_join(consumers, ',')),
       'consumer_id', array_join(consumers, ','), 'Many consumers for one purchase',
       to_json(struct(*))
from invalid_purchase_multi_consumers

union all
select 'fertilizer', cast(fertilizer_id as string),
       concat('purchases=', array_join(purchases, ',')),
       'purchase_id', array_join(purchases, ','), 'Fertilizer tied to multiple purchases',
       to_json(struct(*))
from invalid_fertilizer_multi_purchases

union all
select 'avocado', cast(purchase_id as string),
       concat('purchases=', array_join(consumers, ',')),
       'purchase_id', array_join(consumers, ','), 'Avocado tied to multiple purchases',
       to_json(struct(*))
from invalid_avocado_multi_purchases
;

-- ========================================================
-- 11. Gold Output
-- ========================================================
create or replace temp view gold_output as
select distinct
      con.consumer_id
    , con.sex
    , con.age
    , avo.avocado_bunch_id
    , datediff(avo.sold_date, avo.born_date) as avocado_days_sold
    , avo.avocado_ripe_index
    , datediff(avo.picked_date, avo.born_date) as avocado_days_picked
    , fer.fertilizer_type
from validated_consumer con
left join validated_purchase pur 
    on pur.consumer_id = con.consumer_id
    and pur.purchase_id not in (select purchase_id from invalid_purchase_multi_consumers)
left join validated_avocado avo 
    on avo.purchase_id = pur.purchase_id
    and avo.purchase_id not in (select purchase_id from invalid_avocado_multi_purchases)
left join validated_fertilizer fer 
    on fer.purchase_id = pur.purchase_id
    and fer.fertilizer_id not in (select fertilizer_id from invalid_fertilizer_multi_purchases)
order by con.consumer_id;

select * from gold_output;
Select * from quarantine;